In [1]:
import os
import gc 
gc.collect()

import torch
import lightning as pl
from torchinfo import summary

from lightning.pytorch import loggers as pl_loggers
# from functorch.compile import compiled_function, draw_graph
from lightning.pytorch.profilers import PyTorchProfiler
from lightning.pytorch.callbacks import (
    # DeviceStatsMonitor,
    EarlyStopping,
    # LearningRateMonitor,
    # ModelCheckpoint,
    # ModelPruning,
)
from lightning.pytorch.callbacks.progress import TQDMProgressBar


from config import CONFIG,DEVICE
from model import NanoGPT
from dataum import LitAuthorData,GPT2LitAuthorData

# Auxilary utils
torch.backends.cuda.matmul.allow_tf32=True
torch.set_float32_matmul_precision("medium")
torch.cuda.amp.autocast(enabled=True, dtype=torch.float16)
torch.set_default_device(device=DEVICE)
torch.cuda.empty_cache()
pl.seed_everything(270498);

/tmp/ipykernel_59733/2331140124.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  torch.cuda.amp.autocast(enabled=True, dtype=torch.float16)
Seed set to 270498


In [2]:
# ## Loggers
# logger: pl_loggers.TensorBoardLogger = pl_loggers.TensorBoardLogger(
#     save_dir="logs/", name="nanogpt2", log_graph=True,version= input("Enter name of the experiment: ")
# )

# ## CallBacks
# call_backs = [
#     TQDMProgressBar(refresh_rate=10),
#     ModelCheckpoint(
#         monitor="val/loss",                        # val/loss >> train/loss (to get low loss)
#         dirpath=os.path.join("logs", "chkpoints"),
#         filename="{epoch:02d}",
#         save_top_k=1
#     ),
#     DeviceStatsMonitor(cpu_stats=True),
#     # EarlyStopping(monitor="val/loss",mode='min'),
#     LearningRateMonitor(logging_interval="step"),
# ]

# ## Profilers
# perf_dir = os.path.join(os.getcwd(), "logs", "profiler")
# perf_profiler = PyTorchProfiler(
#     dirpath=perf_dir,
#     filename="perf_logs_pytorch",
#     group_by_input_shapes=True,
#     emit_nvtx=torch.cuda.is_available(),
#     activities=(
#         [
#             torch.profiler.ProfilerActivity.CPU,
#             torch.profiler.ProfilerActivity.CUDA,
#         ]
#         if torch.cuda.is_available()
#         else [
#             torch.profiler.ProfilerActivity.CPU,
#         ]
#     ),
#     schedule=torch.profiler.schedule(
#         wait=1, warmup=1, active=5, repeat=3, skip_first=True
#     ),
#     profile_memory=True,
#     with_stack=True,
#     with_flops=True,
#     with_modules=True,
#     on_trace_ready=torch.profiler.tensorboard_trace_handler(
#         str(os.path.join(perf_dir, "trace"))
#     ),
# )

In [3]:
# Dataset
dm = GPT2LitAuthorData(
    file_path=os.path.join(os.getcwd(), "dataum", f"input.txt.keep"),
    block_size=CONFIG['data'].get('seq_len'),
    batch_size=CONFIG['data'].get('batch_size'),
    num_workers=CONFIG['data'].get("num_workers"),
    encoder='gpt2'
)

dm.prepare_data()
dm.setup()

In [4]:
# NanoGPT Model
model = NanoGPT(
    d_model=CONFIG["model"].get("d_model"),
    seq_len=CONFIG["data"].get("seq_len"),
    vocab_size=dm.train_ds.vocab_size,
    n_head=CONFIG["model"].get("n_head"),
    n_layer=CONFIG["model"].get("n_layer"),
    lr=CONFIG["lr"],
    bias=False,
    dropout_rate=CONFIG["model"].get("dropout"),
)


In [5]:
# CHKPOINT = torch.load(os.path.join(os.getcwd(),"logs","chkpoints",'nanogpt_epoch=01.ckpt'))
CHKPOINT = torch.load(os.path.join(os.getcwd(),"logs","chkpoints",'epoch=04.ckpt'))

/tmp/ipykernel_59733/384950585.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  CHKPOINT = torch.load(os.path.join(os.getcwd(),"logs","chkpoints",'epoch=04.ckpt'))


In [6]:
model.load_state_dict(CHKPOINT['state_dict'])

<All keys matched successfully>

In [7]:
# batch = next(iter(dm.train_dataloader()))
# ip,op = batch
# logger.log_graph(model.to(DEVICE),(ip.to(DEVICE),op.to(DEVICE)))
# with torch.autograd.profiler.profile() as prof:
#     output = model.to(DEVICE)(batch[0].to(DEVICE))

# os.makedirs(name=os.path.join(os.path.dirname(__file__),'logs','profiler'),exist_ok=True)
# with open(os.path.join(os.path.dirname(__file__),'logs','profiler',"cpu_throttle.txt"), "w") as text_file:
#     text_file.write(f"{prof.key_averages().table(sort_by='self_cpu_time_total',top_level_events_only=False)}")


In [8]:
## Model Summary
# summary(
#     model=model,
#     input_data=batch[0],
#     depth=5,
#     verbose=2,
#     col_width=16,
#     col_names=[
#         "input_size",
#         "output_size",
#         "num_params",
#         "kernel_size",
#         "mult_adds",
#     ],
#     row_settings=["var_names"],
# )

In [9]:
trainer = pl.Trainer(
    max_epochs=CONFIG["trainer"].get("epoch"),
    # callbacks=call_backs,
    # logger=logger,
    precision='16-mixed',
    profiler='pytorch',#perf_profiler,#'advanced',
    enable_model_summary=False,
    enable_progress_bar=False,
    enable_checkpointing=False
)

Using 16bit Automatic Mixed Precision (AMP)
/home/muthu/miniconda3/envs/venv/lib/python3.10/site-packages/lightning/pytorch/plugins/precision/amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
# Validating
trainer.validate(model, dataloaders=dm)

Missing logger folder: /home/muthu/GitHub/spaces 🚀/NanoTalker/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/muthu/miniconda3/envs/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
[W723 23:07:19.762330026 collection.cpp:613] Warning: void cutlass::Kernel2<cutlass_75_tensorop_f16_s1688gemm_f16_128x64_tn_align1>(cutlass_75_tensorop_f16_s1688gemm_f16_128x64_tn_align1::Params) (function operator())


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      val/loss_epoch       │     2.617523193359375     │
└───────────────────────────┴───────────────────────────┘

VALIDATE Profiler Report
Profile stats for: records
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         2.13%       3.060ms        82.48%     118.530ms      39.510ms       0.000us         0.00%      79.126ms      26.375ms             3  
                                             aten::item         0.02%      25.767us        47.09%      67.677ms       7.520ms       0.000us         0.00%  

[{'val/loss_epoch': 2.617523193359375}]

In [11]:
### Generation
def generate_text(prompt: str, max_new_token: int, dm=dm, imodel=model,device=DEVICE):
    encoded = dm.train_ds.enc.encode_ordinary(prompt)
    imodel.to(device)
    with torch.no_grad():
        encoded_text = torch.tensor(encoded,device=device).unsqueeze(0)
        new_word_predict = []
        for _ in range(max_new_token):
            encoded_text = encoded_text[:, -32:]
            logits, _ = imodel(encoded_text)
            logits = logits[:, -1, :]
            probs = torch.nn.functional.softmax(logits, dim=-1)
            next_word = torch.multinomial(probs, num_samples=1)
            new_word_predict.append(next_word.item())
            encoded_text = torch.cat((encoded_text[:, imodel.seq_len:], next_word), dim=1)             # Beam Search in Decodiing Strategy
    res = encoded + new_word_predict
    return dm.train_ds.decode(res)

In [17]:
print(generate_text("Let us loves him",30,dm=dm,imodel=model))

Let us loves himury trpot Perever distraught wide lookingopen prosperity imitateuf occ

OrRush autumn infectious spotethetition book arroganceChKE divorced Bad fond statue
